# CAMB-EDE Timing Tests

Notebook for timing and optimizing CAMB-EDE. Author: João Victor S. Rebouças, December 2021.

Vivian's comments:
 - Make a table with: integration parameters, runtime (normalized or difference to wcdm), n of cores, chi^2 (any foreground) for each different dataset (CMB, BAO, SNe)
 - chi^2 difference must be much less than 1
 - Integration parameters: nsteps, zcfdeprec, maybe npointslinear and log_treshold, integrate_tol
 - Close most programs before testing, use htop
 - Do for H0 first, then thetastar
 - Integrate until rho_field/rho_lambda = treshold
 - astart around $10^{-6}$
 - Start with 1 core, then test the scaling in GridUnesp (1, 2, 4, 8 cores)
 - "If you use non-adaptative time steps, you REALLY need to know your system" - Vivian Miranda
 
What I'm investigating: For CAMB-quintessence, quintessence is 2x as slower than lcdm. Why is 2.5x slower in CAMB-EDE?

Note - It takes much longer to fit fde = 0.2 than fde = 0.07;
zc doesn't make much difference

In [1]:
# Vivian's comments:
# "if you use non-adaptive time step, you REALLY need to know your model
# start with one model
# col1 - what i'm trying: zc accuracy, V0 window, binsearch accuracy, number of bg time steps; col2 = chi2 (for each likelihood), col3 = runtime
# do for H0, not thetastar
# probably we will use npoints around a couple hundred
# astart = 10^(-6)
# optimizations in background: accuracy, change time step?,  
# chi^2 for EACH individual likelihoods involved

# compare with literature
# long tails for zc
# reproduce for quintessence
# after: test scaling in gridunesp: use 1, 2, 4, 6, 8 cores and compare
# idea: if the field begins frozen, initial time step can be large. just when the field starts to be dynamic the time step needs to be small.
# adaptive time step!!!!!!!!
# use NAG integrator
# numerical recipes - adaptive runge kutta
# quintessence: one accuracy for z < 10, another for z > 10
# compare chains from old and new code

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import camb
import sys, platform, os
import time
camb_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0,camb_path)
print('Using CAMB %s installed at %s'%(camb.__version__,os.path.dirname(camb.__file__)))
# If you have many CAMB versions at your disposal, it's always useful to check which CAMB is being used.

Using CAMB 1.3.4 installed at /home/joaov/CAMB/CAMB-EDE/camb


In [3]:
# Energy density unit conversion
Mpc = 3.085678e22
c = 2.99792458e8
G = 6.6738e-11
kappa = 8 * np.pi * G
hplanck = 6.62606896e-34
hbar = hplanck/2/np.pi
Tpl = np.sqrt(kappa*hbar/c**5)
energyunitconversion = Mpc**2/(c**2 * Tpl**2)
# Potentials:
rocknroll = 1
axionede = 2
# Repetitions for each try
number_of_reps = 10

## 1 - $\Lambda$CDM

In [4]:
# LambdaCDM
lcdmruntime = 0
for i in range(number_of_reps):
    start = time.time()
    pars = camb.set_params(# Background
                                    thetastar=0.0104, ombh2=0.02238280, omch2=0.1201075, TCMB = 2.7255,                      
                                    # Neutrinos
                                    omnuh2 = 0, num_nu_massless=3.044, num_nu_massive = 0,
                                    nu_mass_degeneracies=[0], nu_mass_numbers = [0],
                                    # Initial Power Spectrum
                                    As = 2.100549e-09, ns = 0.9660499, 
                                    YHe = 0.246, WantTransfer=True)
    lcdmresults = camb.get_results(pars)
    end = time.time()
    lcdmruntime = lcdmruntime + (end-start)
print('Avg runtime for LambdaCDM: %.2f seconds' % (lcdmruntime/number_of_reps))

Avg runtime for LambdaCDM: 0.98 seconds


## 2 - Rock 'n' Roll EDE with zc/fde

In [5]:
# Timing for Rock 'n' Roll w/ zc/fde
ederuntime = 0
for i in range(number_of_reps):
    start = time.time()
    edecosmology = camb.set_params(# Background
                                    thetastar = 0.0104, ombh2=0.02238280, omch2=0.1201075, TCMB = 2.7255,
                                    # EDE with inverse power law potential
                                    dark_energy_model = 'EarlyDarkEnergy', which_potential = 1,
                                    n = 4, use_zc = True, zc = 3000, fde_zc = 0.05,                          
                                    # Neutrinos
                                    omnuh2 = 0, num_nu_massless=3.044, num_nu_massive = 0,
                                    nu_mass_degeneracies=[0], nu_mass_numbers = [0],
                                    # Initial Power Spectrum
                                    As = 2.100549e-09, ns = 0.9660499, 
                                    YHe = 0.246, WantTransfer=True, npoints=200, zcfdeprec=0.01)
    ederesults = camb.get_results(edecosmology)
    end = time.time()
    ederuntime = ederuntime + (end-start)
print('Avg runtime for EDE w/ zc/fde: %.2f seconds' % (ederuntime/number_of_reps))
print('%.2f times slower than lambda-CDM' % (ederuntime/lcdmruntime))

Avg runtime for EDE w/ zc/fde: 2.55 seconds
2.61 times slower than lambda-CDM


## 3 - Rock 'n' Roll EDE without zc/fde

In [16]:
# Timing for Rock 'n' Roll w/out zc/fde
ederuntimenozc = 0
for i in range(number_of_reps):
    start = time.time()
    edecosmology = camb.set_params(# Background
                                    thetastar=0.0104, ombh2=0.02238280, omch2=0.1201075, TCMB = 2.7255,
                                    # EDE with inverse power law potential
                                    dark_energy_model = 'EarlyDarkEnergy', which_potential = 1,
                                    n = 4, use_zc = False, V0 = 20/energyunitconversion, initial_phi = 1,                         
                                    # Neutrinos
                                    omnuh2 = 0, num_nu_massless=3.044, num_nu_massive = 0,
                                    nu_mass_degeneracies=[0], nu_mass_numbers = [0],
                                    # Initial Power Spectrum
                                    As = 2.100549e-09, ns = 0.9660499, 
                                    YHe = 0.246, WantTransfer=True, npoints=400)
    ederesults = camb.get_results(edecosmology)
    end = time.time()
    ederuntimenozc = ederuntimenozc + (end-start)
print('Avg runtime for EDE w/out zc/fde: %.2f seconds' % (ederuntimenozc/number_of_reps))
print('%.2f times slower than lambda-CDM' % (ederuntimenozc/(lcdmruntime)))

Avg runtime for EDE w/out zc/fde: 2.44 seconds
2.32 times slower than lambda-CDM


## 4 - AxionEDE with zc/fde

In [7]:
# Timing for Axion w/ zc/fde
ederuntime = 0
for i in range(number_of_reps):
    start = time.time()
    edecosmology = camb.set_params(# Background
                                    thetastar=0.0104, ombh2=0.02238280, omch2=0.1201075, TCMB = 2.7255,
                                    # EDE with inverse power law potential
                                    dark_energy_model = 'EarlyDarkEnergy', which_potential = 2,
                                    n = 4, use_zc = True, zc = 3000, fde_zc = 0.05,                          
                                    # Neutrinos
                                    omnuh2 = 0, num_nu_massless=3.044, num_nu_massive = 0,
                                    nu_mass_degeneracies=[0], nu_mass_numbers = [0],
                                    # Initial Power Spectrum
                                    As = 2.100549e-09, ns = 0.9660499, 
                                    YHe = 0.246, WantTransfer=True)
    ederesults = camb.get_results(edecosmology)
    end = time.time()
    ederuntime = ederuntime + (end-start)
print('Avg runtime for AxionEDE w/ zc/fde: %.2f seconds' % (ederuntime/number_of_reps))
print('%.2f times slower than lambda-CDM' % (ederuntime/lcdmruntime))

Avg runtime for AxionEDE w/ zc/fde: 2.55 seconds
2.70 times slower than lambda-CDM
